In [1]:
import pandas as pd
import glob
import json

path = './data/saenggak'
files = glob.glob(path + "/*.csv")
li = []

for file in files:
    df = pd.read_csv(file, encoding='CP949')
    li.append(df)

saenggak = pd.concat(li, axis=0, ignore_index=True)
print(saenggak.shape)
saenggak.head()

(231, 20)


,배달구분,진행상황,결제구분,카<->현,남은시간,요청시간,픽업,기사도착,도착지,도착지 전화번호,음식요금,배달요금,픽업기사,기사전화,진행시간,결제요청,카드금액,카드결제,기타,Unnamed: 19
0,배달대행,완료,NaN,NaN,NaN,18:16,18:27,NaN,NaN,NaN,13200,3000,안하영,010-4449-7953,08/13 18:32,선결제,0,미결제[][],문앞에 놓구 벨눌러주세요.,NaN
1,배달대행,완료,NaN,NaN,NaN,18:16,18:28,NaN,NaN,NaN,13700,3000,김범수,010-8984-6264,08/13 18:29,선결제,0,미결제[][],조심히 안전하게 와주세요 :),NaN
2,배달대행,완료,NaN,NaN,NaN,17:53,18:04,NaN,NaN,NaN,13700,3000,김경섭,010-9461-3455,08/13 18:04,선결제,0,미결제[][],조심히 안전하게 와주세요 :),NaN
3,배달대행,완료,NaN,NaN,NaN,17:53,18:13,NaN,NaN,NaN,16800,3500,이문호,010-8424-8227,08/13 18:29,선결제,0,미결제[][],조심히 안전하게 와주세요 :),NaN
4,배달대행,완료,NaN,NaN,NaN,17:47,18:03,NaN,NaN,NaN,17700,3000,오정현,010-6777-2063,08/13 18:14,선결제,0,미결제[][],문 앞에 두고 노크 부탁드립니다!,NaN


In [2]:
saenggak.drop(['Unnamed: 19'], axis=1, inplace=True)
print(saenggak.shape)

(231, 19)


In [3]:
baemin = pd.read_csv('./baemin_compiled.csv')
baemin.head()

,주문번호,수령방법,결제방법,항목,추가선택,드레싱,서비스타입,주문금액,배달팁,결제금액,주문시각,접수시각,배달시각,주소,요청사항,배달요청사항
0,B0KB00H8UL,배달,바로결제,"훈제연어 샐러드, 훈제연어 샐러드",NaN,렌치,배민,18800,2000,20800,2020-01-02 17:58:18,2020-01-02 18:00:26,2020-01-02 18:40:26,전북 전주시 덕진구 금암1동 ****,NaN,[]
1,B0KC005QKF,배달,바로결제,"닭가슴살 샐러드, 훈제연어 샐러드",NaN,"오리엔탈, 렌치",배민,11900,3000,14900,2020-01-03 11:56:51,2020-01-03 11:57:13,2020-01-03 12:37:13,전북 전주시 덕진구 덕진동 ****,NaN,[]
2,B0KC00731E,배달,바로결제,"목살 샐러드, 훈제연어 샐러드",오리엔탈 드레싱,"발사믹, 렌치",배민,16300,1100,17400,2020-01-03 12:29:28,2020-01-03 12:30:13,2020-01-03 13:10:13,전북 전주시 완산구 중앙동 ****,NaN,[]
3,B0KC00AE0R,배달,바로결제,"훈제연어 샐러드, 목살 샐러드","목살 (100g), 숲속토핑 (110g)","렌치, 발사믹",배민,22000,2000,24000,2020-01-03 14:08:40,2020-01-03 14:08:56,2020-01-03 14:58:56,전북 전주시 덕진구 덕진동 ****,콩이 있다면 콩을 빼주시고 올리브 많이 넣어주세요,콩이 있다면 콩을 빼주시고 올리브 많이 넣어주세요
4,B0KC00CGQW,배달,바로결제,훈제연어 샐러드,NaN,렌치,배민,9900,3500,13400,2020-01-03 15:24:54,2020-01-03 15:25:21,2020-01-03 16:15:21,전라북도 전주시 덕진구 인후2동 ****,NaN,전화주시면 바로나갈게요


In [4]:
saenggak_dates = set(map(lambda x: x[:5].replace('/', '-'), saenggak['진행시간']))
saenggak_dates

{'08-12', '08-13', '08-17', '08-18', '08-19', '08-20', '08-25'}

In [11]:
baemin.shape

(3990, 16)

In [5]:
non_matched = baemin.copy()
matched = baemin.copy()
for idx, row in baemin.iterrows():
    if row['주문시각'][5:10] in saenggak_dates:
        non_matched.drop(idx, inplace=True)
    else:
        matched.drop(idx, inplace=True)
print(non_matched.shape)
print(matched.shape)

(3838, 16)
(152, 16)


In [10]:
from datetime import datetime

for idx, row in matched.iterrows():
    # 두 데이터간 '요금' 과 '요청사항' 일치하는 항목의 인덱스(리스트)를 '요금_요청' 칼럼에 추가
    row['요금_요청'] = saenggak.loc[(row['결제금액']==saenggak['음식요금'])&(row['배달요청사항']==saenggak['기타'])].index.values
    print(f"요금요청: {row['요금_요청']}")
    # 요금_요청 칼럼의 매칭된 인덱스 리스트 중에서 시간차가 가장 적은 것을 골라내서 '최소시간차' 칼럼에 하나의 (최적의) 매칭 인덱스 저장
    matched_dt = min([row['주문시각'], row['접수시각']])[5:-3].replace('-', '/')
    tdiff = {}.fromkeys(row['요금_요청'].tolist(), 0)
    print(f"tdiff: {tdiff}")
    
    if len(row['요금_요청'])==1:
        best_match = int(row['요금_요청'])
    else:
        for i in row['요금_요청']:
            saenggak_dt = f"{saenggak.loc[i,'진행시간'][:5]} {saenggak.loc[i, '요청시간']}"
            FMT = '%m/%d %H:%M'
            diff_calculated = abs(datetime.strptime(saenggak_dt, FMT) - datetime.strptime(matched_dt, FMT))
            tdiff[i] = diff_calculated
        best_match = int(min(tdiff))
        matched.loc[idx, '최소시간차'] = best_match
    
    # 실제 배달료 계산
    matched.loc[idx, '실제배달료'] = saenggak.loc[best_match, '배달요금']+300

요금요청: [161]
tdiff: {161: 0}
요금요청: [160]
tdiff: {160: 0}
요금요청: [159]
tdiff: {159: 0}
요금요청: [158]
tdiff: {158: 0}
요금요청: [157]
tdiff: {157: 0}
요금요청: []
tdiff: {}


ValueError: min() arg is an empty sequence

In [ ]:
matched.head()

In [ ]:
saenggak.iloc[[158]]

In [ ]:
# Export
matched.to_csv('./배민+생각_매칭데이터.csv')
non_matched.to_csv('./배민+생각_노매칭데이터.csv')